In [1]:
import sys
sys.path.append('../')

import os
import json
import time

from glob import glob
from tqdm import tqdm
import numpy as np
import pandas as pd

from utils.elastix import excute_cmd, register_elastix, control_points_transformix
from utils.filemanager import create_directory_if_not_exists, replace_text_in_file, add_and_delete_rows, delete_added_rows
from utils.niftimanager import load_nifti, show_nifti
from utils.landmarks import get_landmarks_from_txt, write_landmarks_to_list
from utils.logger import pprint_objects
from utils.metrics import compute_TRE
from utils.utils import format_elapsed_time

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

In [83]:
print(excute_cmd('elastix --help'))

elastix version: 4.700

elastix registers a moving image to a fixed image.
The registration-process is specified in the parameter file.
  --help, -h displays this message and exit
  --version  output version information and exit

Call elastix from the command line with mandatory arguments:
  -f        fixed image
  -m        moving image
  -out      output directory
  -p        parameter file, elastix handles 1 or more "-p"

Optional extra commands:
  -fMask    mask for fixed image
  -mMask    mask for moving image
  -t0       parameter file for initial transform
  -priority set the process priority to high, abovenormal, normal (default),
            belownormal, or idle (Windows only option)
  -threads  set the maximum number of threads of elastix

The parameter-file must contain all the information necessary for elastix to runproperly. That includes which metric to use, which optimizer, which transform, etc. It must also contain information specific for the metric, optimizer, transfo

In [2]:
os.listdir('../niftiData/')

['copd1', 'copd2', 'copd3', 'copd4']

In [3]:
train_path = '../niftiData'

# prepare the paths
exhale_volumes = [path.replace('\\', '/') for path in sorted(glob(os.path.join(train_path, "***" , "*eBHCT.nii.gz"), recursive=True))]
inhale_volumes = [path.replace('\\', '/') for path in sorted(glob(os.path.join(train_path, "***" , "*iBHCT.nii.gz"), recursive=True))]

In [4]:
exhale_volumes

['../niftiData/copd1/copd1_eBHCT.nii.gz',
 '../niftiData/copd2/copd2_eBHCT.nii.gz',
 '../niftiData/copd3/copd3_eBHCT.nii.gz',
 '../niftiData/copd4/copd4_eBHCT.nii.gz']

Register and transform all control points

In [5]:
os.listdir('../elastix-parameters')

['Par0003', 'Par0004', 'Par0007', 'Par0011', 'Par0049']

As we have alot of parameters that are suitable for CT registration. We can experiment them and see which performs the best.

Experimenting Par0003 models with elastix and transformix.

In [6]:
Par_base = '../elastix-parameters/Par0049'
Par_base

'../elastix-parameters/Par0049'

In [7]:
os.listdir(Par_base)

['Par0049_stdT-advanced.txt',
 'Par0049_stdT2000itr.txt',
 'Par0049_stdTL-advanced.txt',
 'Par0049_stdTL.txt']

##### Parameter Model Par0049

In [11]:
# Setting the experiment registration parameter
reg_params = f'-p "{os.path.join(Par_base, "Par0049_stdT2000itr.txt")}"'.replace('\\', '/')
reg_params_key = 'Par0049_stdT2000itr'

print(f"Experimenting using {reg_params} params command...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path in zip(exhale_volumes, inhale_volumes):
    loop_start_time = time.time()

    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../rawData/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from rawData dir
    e_cntl_pt = f'../rawData/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../rawData/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # elastix registration
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        # fMask = test_mask,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # prepare the points file to match transformix description in the manual
    # <index, point>
    # <number of points>
    # point1 x point1 y [point1 z]
    add_and_delete_rows(e_cntl_pt, row1 = '', row2 = ' 300.000000') # row2: number of points (rows)

    # transformix control point transformation
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = e_cntl_pt,
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.0.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # remove the first two rows that were added for transformix in the moving txt file
    delete_added_rows(e_cntl_pt)

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)

    # evaluate
    TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, file_information['voxel_dim'])
    print("Dataset Results:- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}).")
    
    TRE_mean, TRE_std = compute_TRE(i_cntl_pt, output_landmarks_path, file_information['voxel_dim'])
    print("Transformed Results:- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p "../elastix-parameters/Par0049/Par0049_stdT2000itr.txt" params command...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Time for current registration loop: 1 minutes and 19 seconds
Dataset Results:-  (Mean TRE: 26.33) (STD TRE: 11.42).
Transformed Results:-  (Mean TRE: 36.51) (STD TRE: 15.82). 

{'name': 'copd2', 'image_dim': [512, 512, 102], 'voxel_dim': [0.645, 0.645, 2.5], 'features': 612, 'displacement_mean': 21.77, 'displacement_std': 6.46, 'origin': [1, 1, 1]}
Time for current registration loop: 1 minutes and 14 seconds
Dataset Results:-  (Mean TRE: 21.79) (STD TRE: 6.46).
Transformed Results:-  (Mean TRE: 30.41) (STD TRE: 9.07). 

{'name': 'copd3', 'image_dim': [512, 512, 126], 'voxel_dim'

In [12]:
# Setting the experiment registration parameter
reg_params = f'-p "{os.path.join(Par_base, "Par0049_stdT-advanced.txt")}"'.replace('\\', '/')
reg_params_key = 'Par0049_stdT-advanced'

print(f"Experimenting using {reg_params} params command...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path in zip(exhale_volumes, inhale_volumes):
    loop_start_time = time.time()

    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../rawData/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from rawData dir
    e_cntl_pt = f'../rawData/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../rawData/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # elastix registration
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        # fMask = test_mask,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)
    
    # prepare the points file to match transformix description in the manual
    # <index, point>
    # <number of points>
    # point1 x point1 y [point1 z]
    add_and_delete_rows(e_cntl_pt, row1 = '', row2 = ' 300.000000') # row2: number of points (rows)

    # transformix control point transformation
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = e_cntl_pt,
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.0.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # remove the first two rows that were added for transformix in the moving txt file
    delete_added_rows(e_cntl_pt)

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)

    # evaluate
    TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, file_information['voxel_dim'])
    print("Dataset Results:- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}).")
    
    TRE_mean, TRE_std = compute_TRE(i_cntl_pt, output_landmarks_path, file_information['voxel_dim'])
    print("Transformed Results:- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\

overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p "../elastix-parameters/Par0049/Par0049_stdT-advanced.txt" params command...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Time for current registration loop: 9 minutes and 24 seconds
Dataset Results:-  (Mean TRE: 26.33) (STD TRE: 11.42).
Transformed Results:-  (Mean TRE: 42.06) (STD TRE: 21.42). 

{'name': 'copd2', 'image_dim': [512, 512, 102], 'voxel_dim': [0.645, 0.645, 2.5], 'features': 612, 'displacement_mean': 21.77, 'displacement_std': 6.46, 'origin': [1, 1, 1]}
Time for current registration loop: 7 minutes and 40 seconds
Dataset Results:-  (Mean TRE: 21.79) (STD TRE: 6.46).
Transformed Results:-  (Mean TRE: 34.65) (STD TRE: 10.94). 

{'name': 'copd3', 'image_dim': [512, 512, 126], 'voxel_d